# ANNUITY CALCULATOR

In this notebook I use the simple mortgage calculator to generate mortgage cashflows.

In [2]:
from financepy.products.bonds import *
from financepy.finutils import *
from financepy.products.libor import *

####################################################################
# FINANCEPY BETA Version 0.180 - This build:  22 Sep 2020 at 22:29 #
#      This software is distributed FREE & WITHOUT ANY WARRANTY    #
# For info and disclaimer - https://github.com/domokane/FinancePy  #
#    For bug reports and comments - allmypythonprojects@gmail.com  #
####################################################################



## CREATE AN ANNUITY STREAM

Calculate the cashflows on an annuity

In [3]:
settlementDate = FinDate(20, 6, 2018)

In [4]:
maturityDate = settlementDate.addTenor("5Y")  # FinDate(20, 9, 2028)
coupon = 0.05
frequencyType = FinFrequencyTypes.SEMI_ANNUAL
calendarType = FinCalendarTypes.WEEKEND
busDayAdjustType = FinBusDayAdjustTypes.FOLLOWING
dateGenRuleType = FinDateGenRuleTypes.BACKWARD
basisType = FinDayCountTypes.ACT_360
face = ONE_MILLION

In [5]:
annuity = FinBondAnnuity(maturityDate,coupon,frequencyType,calendarType,
                         busDayAdjustType,dateGenRuleType,basisType,face)

In [6]:
print(annuity)

OBJECT TYPE: FinBondAnnuity
MATURITY DATE: TUE 20 JUN 2023
FREQUENCY: FinFrequencyTypes.SEMI_ANNUAL
CALENDAR: FinCalendarTypes.WEEKEND
BUS_DAY_RULE: FinBusDayAdjustTypes.FOLLOWING
DATE_GEN_RULE: FinDateGenRuleTypes.BACKWARD



In [7]:
annuity.printFlows(settlementDate)

THU 20 DEC 2018 , 25416.666666666668
THU 20 JUN 2019 , 25277.777777777777
FRI 20 DEC 2019 , 25416.666666666668
SAT 20 JUN 2020 , 25416.666666666668
SUN 20 DEC 2020 , 25416.666666666668
SUN 20 JUN 2021 , 25277.777777777777
MON 20 DEC 2021 , 25416.666666666668
MON 20 JUN 2022 , 25277.777777777777
TUE 20 DEC 2022 , 25416.666666666668
TUE 20 JUN 2023 , 25277.777777777777


## We can value the flows by discounting them

First we build a Libor swap curve

In [8]:
depos = []
dcType = FinDayCountTypes.ACT_360
fixedFreq = FinFrequencyTypes.SEMI_ANNUAL
swapType = FinLiborSwapTypes.PAYER
swap1 = FinLiborSwap(settlementDate,"1Y",swapType,0.0500,fixedFreq,dcType)
swap2 = FinLiborSwap(settlementDate,"2Y",swapType,0.0500,fixedFreq,dcType)
swap3 = FinLiborSwap(settlementDate,"3Y",swapType,0.0500,fixedFreq,dcType)
swap4 = FinLiborSwap(settlementDate,"4Y",swapType,0.0500,fixedFreq,dcType)
swap5 = FinLiborSwap(settlementDate,"5Y",swapType,0.0500,fixedFreq,dcType)
swaps = [swap1, swap2, swap3, swap4, swap5]

liborCurve = FinLiborCurve(settlementDate, depos, [], swaps)

Here are the flows on the fixed leg of the 5Y swap

In [8]:
swap5.printFixedLegFlows()

START DATE: WED 20 JUN 2018
MATURITY DATE: TUE 20 JUN 2023
COUPON (%): 5.0
FIXED LEG FREQUENCY: FinFrequencyTypes.SEMI_ANNUAL
FIXED LEG DAY COUNT: FinDayCountTypes.ACT_360
PAYMENT_DATE     YEAR_FRAC        FLOW
THU 20 DEC 2018   0.50833333         2.54
THU 20 JUN 2019   0.50555556         2.53
FRI 20 DEC 2019   0.50833333         2.54
MON 22 JUN 2020   0.51388889         2.57
MON 21 DEC 2020   0.50555556         2.53
MON 21 JUN 2021   0.50555556         2.53
MON 20 DEC 2021   0.50555556         2.53
MON 20 JUN 2022   0.50555556         2.53
TUE 20 DEC 2022   0.50833333         2.54
TUE 20 JUN 2023   0.50555556         2.53


In [9]:
cleanValue = annuity.cleanPriceFromDiscountCurve(settlementDate, liborCurve)

In [10]:
dirtyValue = annuity.fullPriceFromDiscountCurve(settlementDate, liborCurve)

The price of the annuity is quoted based on a par notional

In [11]:
print("CLEAN PRICE:", cleanValue)
print("DIRTY PRICE:", dirtyValue)
print("ACCRD VALUE:", annuity._accruedInterest)

CLEAN PRICE: 22.154860612058915
DIRTY PRICE: 22.154860612058915
ACCRD VALUE: 0.0


Copyright (c) 2020 Dominic O'Kane